# IKT457 — Assignment 2: Tsetlin Machine (Breast Cancer Rules)

**Student:** _<your name here>_  
**Course:** IKT457  
**Notebook generated:** 2025-09-19 07:00:54

This notebook follows the assignment instructions:

**Task 1**
1. Use the Jupyter Notebook for Chapter 1 as a starting point (conceptual inspiration).
2. Create a dataset for the 6 breast cancer patients.
3. Add the three breast cancer recurrence rules **R1**, **R2**, and **R3** manually (from slides).
4. Classify all 6 patients with the three rules.

**Task 2**
- Mix Type I and Type II Feedback with forget value **0.8** and memorize value **0.2** to learn a rule for **Recurrence**.
- Mix Type I and Type II Feedback with forget value **0.8** and memorize value **0.2** to learn a rule for **Non‑recurrence**.

**Task 3**
- Repeat Task 2 with values **(forget=0.5, memorize=0.5)** and **(forget=0.2, memorize=0.8)**.

The code is intentionally simple and self‑contained to make the learning dynamics transparent.

In [53]:
# --- Setup & Imports ---
import random
from random import choice
from pprint import pprint

# Deterministic runs for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

## Data: 6 Patients (3 Recurrence, 3 Non-recurrence)

We encode each patient with boolean literals. A literal named `NOT x` denotes the negation of feature `x`.

In [54]:
# Recurrence (cancer) patients
cancer_patients = [
    {"lt40": False, "ge40": True,  "premono": False,
     "zero_two": False, "three_five": True, "six_eight": False,
     "deg_malig_1": False, "deg_malig_2": False, "deg_malig_3": True},

    {"lt40": False, "ge40": True,  "premono": False,
     "zero_two": False, "three_five": False, "six_eight": True,
     "deg_malig_1": False, "deg_malig_2": False, "deg_malig_3": True},

    {"lt40": False, "ge40": False, "premono": True,
     "zero_two": True, "three_five": False, "six_eight": False,
     "deg_malig_1": False, "deg_malig_2": False, "deg_malig_3": True},
]

# Non-recurrence (cancer-free) patients
cancer_free_patients = [
    {"lt40": True,  "ge40": False, "premono": False,
     "zero_two": True, "three_five": False, "six_eight": False,
     "deg_malig_1": False, "deg_malig_2": False, "deg_malig_3": True},

    {"lt40": False, "ge40": True,  "premono": False,
     "zero_two": True, "three_five": False, "six_eight": False,
     "deg_malig_1": False, "deg_malig_2": True,  "deg_malig_3": False},

    {"lt40": False, "ge40": False, "premono": True,
     "zero_two": True, "three_five": False, "six_eight": False,
     "deg_malig_1": True,  "deg_malig_2": False, "deg_malig_3": False},
]
print("Patients with Recurrence (3):")
pprint(cancer_patients)
print("\nPatients with Non-recurrence (3):")
pprint(cancer_free_patients)

Patients with Recurrence (3):
[{'deg_malig_1': False,
  'deg_malig_2': False,
  'deg_malig_3': True,
  'ge40': True,
  'lt40': False,
  'premono': False,
  'six_eight': False,
  'three_five': True,
  'zero_two': False},
 {'deg_malig_1': False,
  'deg_malig_2': False,
  'deg_malig_3': True,
  'ge40': True,
  'lt40': False,
  'premono': False,
  'six_eight': True,
  'three_five': False,
  'zero_two': False},
 {'deg_malig_1': False,
  'deg_malig_2': False,
  'deg_malig_3': True,
  'ge40': False,
  'lt40': False,
  'premono': True,
  'six_eight': False,
  'three_five': False,
  'zero_two': True}]

Patients with Non-recurrence (3):
[{'deg_malig_1': False,
  'deg_malig_2': False,
  'deg_malig_3': True,
  'ge40': False,
  'lt40': True,
  'premono': False,
  'six_eight': False,
  'three_five': False,
  'zero_two': True},
 {'deg_malig_1': False,
  'deg_malig_2': True,
  'deg_malig_3': False,
  'ge40': True,
  'lt40': False,
  'premono': False,
  'six_eight': False,
  'three_five': False,
  'zer

## Rule Evaluation Helper

A *condition* is a list of literals like `["deg_malig_3", "NOT lt40"]`.  
The function below checks whether an observation satisfies all literals.

In [55]:
def evaluate_condition(observation, condition):
    """Return True iff observation satisfies all literals in condition."""
    for feature in observation:
        # Positive literal must be True
        if feature in condition and observation[feature] is False:
            return False
        # Negative literal must be False
        if ('NOT ' + feature) in condition and observation[feature] is True:
            return False
    return True

## Manual Rules R1, R2, R3 (from slides)

- **R1** → Recurrence  
- **R2** → Recurrence  
- **R3** → Non‑recurrence

> Note: In the provided slide set, R1 and R2 can be identical (representing two positive "votes").  
> If your slide shows different literals for R2, update it here.

In [56]:
# Example from your solution (R1 and R2 identical):
R1 = ["deg_malig_3", "NOT lt40"]   # -> Recurrence
R2 = ["deg_malig_3", "NOT lt40"]   # -> Recurrence
R3 = ["zero_two"]                  # -> Non-recurrence

## Task 1: Classify All 6 Patients Using R1, R2, R3

We do a simple voting: each satisfied recurrence rule gives +1 vote, each satisfied non‑recurrence rule gives −1.  
**Tie-breaking:** we treat strictly positive totals as *Recurrence*. This avoids classifying a patient as Recurrence when no rule fires.

In [57]:
def classify_with_manual_rules(observation, R1, R2, R3):
    vote_sum = 0
    if evaluate_condition(observation, R1): vote_sum += 1
    if evaluate_condition(observation, R2): vote_sum += 1
    if evaluate_condition(observation, R3): vote_sum -= 1
    return "Recurrence" if vote_sum > 0 else "Non-recurrence"

print("### Results on 6 patients with (R1,R2,R3) \n")

for i in range(3):
    print(f"Recurrence patient {i+1}:",
          classify_with_manual_rules(cancer_patients[i], R1, R2, R3))
for i in range(3):
    print(f"Non-recurrence patient {i+1}:",
          classify_with_manual_rules(cancer_free_patients[i], R1, R2, R3))

### Results on 6 patients with (R1,R2,R3) 

Recurrence patient 1: Recurrence
Recurrence patient 2: Recurrence
Recurrence patient 3: Recurrence
Non-recurrence patient 1: Non-recurrence
Non-recurrence patient 2: Non-recurrence
Non-recurrence patient 3: Non-recurrence


## A Simple Memory Model (Clause-like)

We emulate the effect of Tsetlin clauses by maintaining a *score* for each literal.  
- Scores are in `[1, 10]` for stability.  
- A learned **condition** consists of all literals with score ≥ 6 (you can tweak the threshold).

**Type I feedback:** Reinforces literals that match the current observation (and penalizes others with forgetting).  
**Type II feedback:** If the current condition fires on a negative example, it strengthens negations to suppress it.

In [58]:
class Memory:
    def __init__(self, forget_value, memorize_value, memory):
        self.memory = memory
        self.forget_value = forget_value
        self.memorize_value = memorize_value

    def get_memory(self):
        return self.memory

    def get_literals(self):
        return list(self.memory.keys())

    def get_condition(self, threshold=6):
        return [lit for lit,score in self.memory.items() if score >= threshold]

    def memorize(self, literal):
        if random.random() <= self.memorize_value and self.memory[literal] < 10:
            self.memory[literal] += 1

    def forget(self, literal):
        if random.random() <= self.forget_value and self.memory[literal] > 1:
            self.memory[literal] -= 1

    def memorize_always(self, literal):
        if self.memory[literal] < 10:
            self.memory[literal] += 1

def fresh_memory(init_score=5):
    feats = ["lt40","ge40","premono","zero_two","three_five","six_eight",
             "deg_malig_1","deg_malig_2","deg_malig_3"]
    mem = {f:init_score for f in feats}
    mem.update({f"NOT {f}":init_score for f in feats})
    return mem

In [59]:
def type_i_feedback(observation, memory: Memory):
    """Reinforce matching literals; softly forget the rest."""
    remaining = set(memory.get_literals())
    for feature, val in observation.items():
        if val:
            memory.memorize(feature)
            remaining.discard(feature)
        else:
            neg = "NOT " + feature
            memory.memorize(neg)
            remaining.discard(neg)
    for lit in remaining:
        memory.forget(lit)

def type_ii_feedback(observation, memory: Memory):
    """If current condition fires but should not, push opposing literals."""
    if evaluate_condition(observation, memory.get_condition()):
        for feature, val in observation.items():
            if not val:
                memory.memorize_always(feature)
            else:
                memory.memorize_always("NOT " + feature)

## Training Utilities

We learn **one clause** for Recurrence and **one clause** for Non‑recurrence by mixing Type I/II feedback on examples.

In [60]:
def classify_patient(observation, pos_rules, neg_rules, threshold=6):
    vote_sum = 0
    for pos_rule in pos_rules:
        if evaluate_condition(observation, pos_rule.get_condition(threshold)): vote_sum += 1
    for neg_rule in neg_rules:
        if evaluate_condition(observation, neg_rule.get_condition(threshold)): vote_sum -= 1
    return "Recurrence" if vote_sum > 0 else "Non-recurrence"

def train(cancer_patients, cancer_free_patients, cancer_rule, cancer_free_rule, iterations=100):
    # Learn recurrence clause
    for _ in range(iterations):
        idx = choice([0,1,2])
        y = choice([0,1])  # 1 => recurrence example, 0 => non-recurrence
        if y == 1:
            type_i_feedback(cancer_patients[idx], cancer_rule)
        else:
            type_ii_feedback(cancer_free_patients[idx], cancer_rule)

    # Learn non-recurrence clause
    for _ in range(iterations):
        idx = choice([0,1,2])
        y = choice([0,1])  # 1 => non-recurrence example, 0 => recurrence
        if y == 1:
            type_i_feedback(cancer_free_patients[idx], cancer_free_rule)
        else:
            type_ii_feedback(cancer_patients[idx], cancer_free_rule)

def evaluate_all(cancer_rule, cancer_free_rule, threshold=6):
    correct = 0
    for j in range(3):
        if classify_patient(cancer_patients[j],[cancer_rule],[cancer_free_rule],threshold) == "Recurrence":
            correct += 1
        if classify_patient(cancer_free_patients[j],[cancer_rule],[cancer_free_rule],threshold) == "Non-recurrence":
            correct += 1
    return correct, 6  # total samples

## Experiments (Tasks 2 & 3)

We run the learning with three (forget, memorize) settings and report the learned conditions and accuracy across the 6 patients.

In [61]:
def run_experiment(forget, memorize, repetitions=50, iters_per_rep=100, threshold=6):
    cancer_rule = Memory(forget, memorize, fresh_memory())
    cancer_free_rule = Memory(forget, memorize, fresh_memory())
    # Multiple short training bursts improve stability while keeping it simple
    for _ in range(repetitions):
        train(cancer_patients, cancer_free_patients, cancer_rule, cancer_free_rule, iterations=iters_per_rep)

    correct, total = evaluate_all(cancer_rule, cancer_free_rule, threshold)
    print(f"""Params: forget={forget}, memorize={memorize}
  Learned Recurrence condition:     {cancer_rule.get_condition(threshold)}
  Learned Non-recurrence condition: {cancer_free_rule.get_condition(threshold)}
  Accuracy: {correct}/{total} = {correct/total:.2f}
""")
    return cancer_rule, cancer_free_rule

print("### Task 2: forget=0.8, memorize=0.2")
_ = run_experiment(0.8, 0.2)

print("### Task 3a: forget=0.5, memorize=0.5")
_ = run_experiment(0.5, 0.5)

print("### Task 3b: forget=0.2, memorize=0.8")
_ = run_experiment(0.2, 0.8)

### Task 2: forget=0.8, memorize=0.2
Params: forget=0.8, memorize=0.2
  Learned Recurrence condition:     ['deg_malig_3', 'NOT lt40', 'NOT deg_malig_1', 'NOT deg_malig_2']
  Learned Non-recurrence condition: ['zero_two', 'NOT three_five', 'NOT six_eight', 'NOT deg_malig_3']
  Accuracy: 6/6 = 1.00

### Task 3a: forget=0.5, memorize=0.5
Params: forget=0.5, memorize=0.5
  Learned Recurrence condition:     ['ge40', 'deg_malig_3', 'NOT lt40', 'NOT premono', 'NOT zero_two', 'NOT three_five', 'NOT six_eight', 'NOT deg_malig_1', 'NOT deg_malig_2']
  Learned Non-recurrence condition: ['zero_two', 'NOT lt40', 'NOT ge40', 'NOT premono', 'NOT three_five', 'NOT six_eight', 'NOT deg_malig_1', 'NOT deg_malig_2', 'NOT deg_malig_3']
  Accuracy: 3/6 = 0.50

### Task 3b: forget=0.2, memorize=0.8
Params: forget=0.2, memorize=0.8
  Learned Recurrence condition:     ['ge40', 'premono', 'zero_two', 'three_five', 'six_eight', 'deg_malig_3', 'NOT lt40', 'NOT ge40', 'NOT premono', 'NOT zero_two', 'NOT three_fiv

## Discussion

- Increasing **memorize** relative to **forget** tends to produce **larger conditions** (more literals retained),
  which can overfit on tiny datasets. Conversely, high **forget** prunes more aggressively.
- Because we use a single clause per class and a tiny dataset (3+3 samples),
  results vary slightly with randomness; setting a **seed** helps reproducibility.
- You can adjust the threshold in `get_condition(threshold=...)` to make the learned clause stricter/looser.

> If your slide specifies a different **R2** than **R1**, update `R2` in the rules cell and re-run the Task 1 block.